In [49]:
import numpy as np
import pandas as pd
#import scipy
from datetime import date, datetime

In [50]:

users = pd.read_csv(r"D:\dataset\users_cleaned.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")
ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv")


In [58]:
#GENERATE USER VECTOR

#change days_spent_watching to a range: check if above certain amount and put a number for the vector
#check if user_completed is above certain amount
#location?

#vector_uf = 

uf = users.loc[:, ["user_id", "user_completed", "user_days_spent_watching", "gender", "location", "birth_date", "stats_episodes"]]
uf["birth_date"] = pd.to_datetime(uf["birth_date"])

#Only include users who have completed at least 5 anime
uf = uf[uf["user_completed"] >= 5]

#Convert birth years to generations
def get_birth_year(date):
    birth_year = date.strftime("%Y")
    return int(birth_year)

uf["birth_date"] = uf["birth_date"].apply(get_birth_year)

uf = uf.rename(columns={"birth_date":"generation"})

#Assign generations based on birth year
generations = [
    (uf["generation"] >= 2013) & (uf["generation"] <= 2025),
    (uf["generation"] >= 1995) & (uf["generation"] <= 2012),
    (uf["generation"] >= 1980) & (uf["generation"] <= 1994),
    (uf["generation"] >= 1965) & (uf["generation"] <= 1979)
]

gen_values = [
    "Gen Alpha",
    "Gen Z",
    "Millenials",
    "Gen X"
]
uf["generation"] = np.select(generations, gen_values, default=uf["generation"])

#Cast days_watched to integers so that users are not missed in conditions
uf["user_days_spent_watching"] = uf["user_days_spent_watching"].astype("int")
#Assign experience levels based on days watched
user_exp = [
    (uf["user_days_spent_watching"] >= 0) & (uf["user_days_spent_watching"] <= 5),
    (uf["user_days_spent_watching"] >= 6) & (uf["user_days_spent_watching"] <= 50),
    (uf["user_days_spent_watching"] >= 51) & (uf["user_days_spent_watching"] <= 99),
    (uf["user_days_spent_watching"] >= 100)
]

exp_values = [
    "Newbie",
    "Regular",
    "Active",
    "Veteran"
]

uf["user_days_spent_watching"] = np.select(user_exp, exp_values, default=uf["user_days_spent_watching"])
uf = uf.rename(columns={"user_days_spent_watching":"experience"})
uf

#rewatched/completed         dropped/completed
#check ratings table and find users most loved and hated genres
#find users that watched the same anime and check ratings

#anime vector
#consider the length of the anime. Ranges: short, medium, long

,user_id,user_completed,experience,gender,location,generation,stats_episodes
0,2255153,49,Active,Female,"Chennai, India",Millenials,3391
1,37326,195,Active,Male,"Detroit,Michigan",Millenials,4903
2,228342,414,Veteran,Male,"Nayarit, Mexico",Millenials,9701
3,327311,5,Regular,Female,"Malaysia, Kuantan",Gen Z,697
4,5015094,114,Regular,Male,"Nijmegen, Nederland",Gen Z,1847
...,...,...,...,...,...,...,...
108706,797785,213,Active,Male,Poland:,Gen Z,3939
108707,1396285,116,Regular,Female,"nhollywood, california",Gen Z,2537
108708,2478991,103,Regular,Female,Belgium,Gen Z,2869
108709,3975907,239,Active,Male,"Latvia, Riga",Gen Z,4469
